In [ ]:
import cv2
from tools_class_new import *
import threading
import numpy as np 
import matplotlib.pyplot as plt
import pyttsx3
import sys
import re
#import pyaudio
#import wave
#import matplotlib.pyplot as plt 
# import sys
# import time
# import re

class Programm:

    def __init__(self, video, line_color, total_frames):
        self.res_text = 0
        self.prev_text = 0
        self.video = video
        self.line_color = line_color
        self.total_frames = total_frames

    # Video-Thread
    def video_thread(self):
        
        # init class/values for while loop
        direct = FindDirection()
        
        #loop through all frames of video
        while self.video.isOpened():
            #Read a single frame
            ret, frame = self.video.read()
            if not ret:
                print("Error load frame")
                break
            
            #Create a Frame Object
            frame = FrameObject(frame)
            #get image from FrameObject
            img = frame.img

            #Create Segmemtation Object  
            seg = Segmentation(frame, self.line_color)
            #get the orientation line
            img_seg = seg.get_orientation_lines()

            frame.build_center_line(img_seg)
            #used later to find the directions
            img = frame.make_block_over_center_line()
            
            # Get Values from FrameObject to Direction Object
            direct.get_values_from_frame_object(frame)
            direct.wait_for_complete_contour(img)
            direct.find_nearest_line(img_seg)
            direct.check_where_to_go()
            res_text = direct.smooth_output()
            self.res_text = res_text
            frame.text_in_frame(res_text)
            frame_show = frame.overlay_segmentation(img_seg)
            

            cv2.imshow('Processed Frame',frame_show)

            # Check for key press to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if video.get(cv2.CAP_PROP_POS_FRAMES) == self.total_frames:
                break
        # Release the video objects and close the windows
        video.release()
        cv2.destroyAllWindows()


    # Audio-Thread
    def audio_thread(self):

        while video.isOpened():
            # Read a single frame
            ret, frame = video.read()
            if not ret:
                print("Error load frame")
                break
            
            # Überprüfen, ob sich das letzte Element geändert hat
            if self.res_text != self.prev_text:
                self.prev_text = self.res_text
                print("audio_txt", self.prev_text)
                audio_output(self.prev_text)
            else:
                print("prev_txt", self.prev_text)
            #time.sleep(0.1)


        # Check for key press to exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if video.get(cv2.CAP_PROP_POS_FRAMES) == total_frames:
                break
        # Release the video objects and close the windows
        video.release()
        cv2.destroyAllWindows()


# Open the video file
video, line_color, total_frames = user_input()

#Instanz of class
Threads = Programm(video, line_color, total_frames)

# Threads starten
video_thread = threading.Thread(target=Threads.video_thread)
audio_thread = threading.Thread(target=Threads.audio_thread)

video_thread.start()
audio_thread.start()

video_thread.join()
audio_thread.join()